### Data Understanding

In [1]:
#Exploratory 
import pandas as pd
import numpy as np

#Data Visualization 
import seaborn as sns
import folium

#Data Modeling
import nltk
from nltk.corpus import gutenberg, stopwords
from nltk.collocations import *
from nltk import FreqDist
from nltk import word_tokenize
import string
import re

#Data Evaluation 


#Ignore any warnings
import warnings;
warnings.filterwarnings('ignore')


In [2]:
data = pd.read_csv('../Data/20191031__30k_data.csv')
data

,Job Title,Job Description,Job Type,Categories,Location,City,State,Country,Zip Code,Address,...,Employer Phone,Employer Logo,Companydescription,Employer Location,Employer City,Employer State,Employer Country,Employer Zip Code,Uniq Id,Crawl Timestamp
0,Shift Manager,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,Del Taco is an American quick service restaura...,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,511f9a53920f4641d701d51d3589349f,2019-08-24 09:13:18 +0000
1,Operations Support Manager,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Atlanta, GA 30342",Atlanta,GA,United States,30342,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,"Based in Atlanta, FOCUS Brands Inc. is an inno...",NaN,NaN,NaN,United States,NaN,4955daf0a3facbe2acb6c429ba394e6d,2019-09-19 08:16:55 +0000
2,Senior Product Manager - Data,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Chicago, IL",Chicago,IL,United States,NaN,NaN,...,NaN,NaN,Vibes Corp. reputation was built and establish...,NaN,NaN,NaN,United States,NaN,a0e0d12df1571962b785f17f43ceae12,2019-09-18 02:13:10 +0000
3,Part-Time Office Concierge,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Festus, MO",Festus,MO,United States,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN,56e411fd731f76ac916bf4fb169250e9,2019-10-24 16:39:13 +0000
4,Print & Marketing Associate,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Cedar Rapids, IA 52404",Cedar Rapids,IA,United States,52404,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,"Staples is The Worklife Fulfillment Company, h...","Cedar Rapids, IA 52404",Cedar Rapids,IA,United States,52404,3fff5c0ad6981bf4bff6260bd5feab63,2019-08-24 22:29:10 +0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,Bilingual Teller,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Lakewood, CO 80226",Lakewood,CO,United States,80226,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,Moneytree is a family-run business that has be...,NaN,NaN,NaN,United States,NaN,db18d0e2de28df6e1e605a3800ffd574,2019-10-25 22:42:37 +0000
29998,"Rental Consultant - Harrison, OH","<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Harrison, OH 45030",Harrison,OH,United States,45030,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN,47f8d7f320223b6f043ef5c345676902,2019-10-25 16:56:18 +0000
29999,Product Support Specialist 5,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Melbourne, FL 32940",Melbourne,FL,United States,32940,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN,d2a18fb9f52495aa7876d4470e5acc61,2019-10-25 01:39:48 +0000
30000,Marketing Strategist (Contract),"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"San Diego, CA",San Diego,CA,United States,NaN,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,Ajinomoto Bio-Pharma Services is a fully integ...,NaN,NaN,NaN,United States,NaN,ce699d2e94a52dfebb13bcf70ed5ffd7,2019-08-23 15:36:47 +0000


In [3]:
data.Companydescription.isnull().sum()

10340

In [4]:
data['Job Description'][1]

'<div id="jobDescriptionText" class="jobsearch-jobDescriptionText">\n<div>\n<p><b>JOB PURPOSE:</b> This position is responsible for providing Operations support to Restaurants related to technology products utilized in the restaurants, which are administered/maintained by the FOCUS Brands Retail Technology Department. This role will act as the primary escalation point for Restaurants, Field Management, and Home Office Teams for the operational use of restaurant systems.</p>\n<p></p>\n<p><b>CORE JOB FUNCTIONS/RESPONSIBILITIES:\n</b></p>\n<ul>\n<li>Program, Procedure, Execution Material Development - Use strategic thinking to identify areas of improvement within the restaurant systems, used by Home Office and restaurant operations. Develop, test, train, integrate, introduce, and execute new and efficient programs/procedures in the following areas including, but not limited to:\n<ul>\n<li>Information Technology/Innovation Projects and Initiatives</li>\n<li>Subject Matter Expertise for day

### Data Preparation

In [5]:
#conda create -n nlp-env numpy=1.19.6 pip wordcloud matplotlib nltk pandas scikit-learn